<a href="https://colab.research.google.com/github/nurniahamid/movie_recommender/blob/main/movie_recomendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Pertanyaan Bisnis**

1. Bagaimana memberikan rekomendasi film yang relevan kepada pengguna berdasarkan preferensi genre yang dimilikinya?

2. Bagaimana menyarankan film kepada pengguna dengan mempertimbangkan perilaku pengguna lain yang memiliki preferensi serupa?


Goals

1. Membangun sistem rekomendasi berbasis konten (content-based filtering) yang dapat menganalisis informasi film (seperti genre) untuk memberikan saran yang sesuai dengan riwayat tontonan pengguna.

2. Mengembangkan sistem rekomendasi berbasis kolaboratif (collaborative filtering) yang dapat memberikan saran kepada pengguna berdasarkan pola rating dan preferensi dari pengguna lain yang serupa.

In [1]:
import pandas as pd

# **Data Understanding**

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nurniahamid","key":"f5ac50c26628fb88a41f246b68d97d62"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d gargmanas/movierecommenderdataset
!unzip movierecommenderdataset.zip -d movierecommenderdataset

Dataset URL: https://www.kaggle.com/datasets/gargmanas/movierecommenderdataset
License(s): GPL-2.0
  0% 0.00/846k [00:00<?, ?B/s]
100% 846k/846k [00:00<00:00, 945MB/s]
Archive:  movierecommenderdataset.zip
  inflating: movierecommenderdataset/movies.csv  
  inflating: movierecommenderdataset/ratings.csv  


In [4]:
movies = pd.read_csv('movierecommenderdataset/movies.csv')
ratings = pd.read_csv('movierecommenderdataset/ratings.csv')

print('Jumlah data film: ', len(movies.movieId.unique()))
print('Jumlah data penilaian yang diberikan pengguna: ', len(ratings.movieId.unique()))

Jumlah data film:  9742
Jumlah data penilaian yang diberikan pengguna:  9724


Variabel dalam data :
- Movies : merupakan jumlah data film
- rating : merupakan data rating pengguna terhadap film

# **Univariate Exploratory Data Analysis**

In [5]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [6]:
#variabel movies
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


melihat ada berapa banyak data berdasarkan movieId dan untuk mengetahui jenis-jenis genre yang ada dalam data

In [7]:
print('Banyak Data : ', len(movies.movieId.unique()))
print('Jumlah Jenis Genre film : ', len(movies.genres.unique()))
print('Jenis Genre film: ', movies.genres.unique())

Banyak Data :  9742
Jumlah Jenis Genre film :  951
Jenis Genre film:  ['Adventure|Animation|Children|Comedy|Fantasy'
 'Adventure|Children|Fantasy' 'Comedy|Romance' 'Comedy|Drama|Romance'
 'Comedy' 'Action|Crime|Thriller' 'Adventure|Children' 'Action'
 'Action|Adventure|Thriller' 'Comedy|Horror'
 'Adventure|Animation|Children' 'Drama' 'Action|Adventure|Romance'
 'Crime|Drama' 'Drama|Romance' 'Action|Comedy|Crime|Drama|Thriller'
 'Comedy|Crime|Thriller' 'Crime|Drama|Horror|Mystery|Thriller'
 'Drama|Sci-Fi' 'Children|Drama' 'Adventure|Drama|Fantasy|Mystery|Sci-Fi'
 'Mystery|Sci-Fi|Thriller' 'Children|Comedy' 'Drama|War'
 'Action|Crime|Drama' 'Action|Adventure|Fantasy' 'Comedy|Drama|Thriller'
 'Mystery|Thriller' 'Animation|Children|Drama|Musical|Romance'
 'Crime|Mystery|Thriller' 'Adventure|Drama' 'Drama|Thriller'
 'Comedy|Crime' 'Action|Sci-Fi|Thriller' 'Action|Comedy|Horror|Thriller'
 'Comedy|Drama' 'Documentary' 'Action|Crime|Drama|Thriller'
 'Crime|Drama|Romance' 'Action|Adventure|Dram

In [8]:
# variabel ratings
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [9]:
print('Banyaknya data rating : ', len(ratings.userId.unique()))
print('Banyaknya film yang di rating : ', len(ratings.movieId.unique()))

Banyaknya data rating :  610
Banyaknya film yang di rating :  9724


In [10]:
ratings.describe()

,userId,movieId,rating,timestamp
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


# **Data Preprocessing**

#### Menggabungkan data movies dan ratings

In [11]:
merged_df = pd.merge(ratings, movies, on='movieId', how='left')
merged_df

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
...,...,...,...,...,...,...
100831,610,166534,4.0,1493848402,Split (2017),Drama|Horror|Thriller
100832,610,168248,5.0,1493850091,John Wick: Chapter Two (2017),Action|Crime|Thriller
100833,610,168250,5.0,1494273047,Get Out (2017),Horror
100834,610,168252,5.0,1493846352,Logan (2017),Action|Sci-Fi


In [12]:
merged_df.isnull().sum()

,0
userId,0
movieId,0
rating,0
timestamp,0
title,0
genres,0


In [13]:
merged_df.duplicated().sum()

np.int64(0)

mempertahankan genre multi-kata dalam kolom genre agar tidak dianggap sebagai dua genre yang berbeda, jadi kita akan mengganti tanda (-) menjadi (_) sebelum vektorisasi dan mengubah Kolom genres dalam format Adventure|Animation|Children menjadi Adventure Animation Children atau mengubah tanda pemisal menjadi spasi

In [14]:
merged_df['genres'] = (
    merged_df['genres']
    .str.lower()
    .str.replace('-', '_')
    .str.replace('|', ' ')
)
merged_df

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),adventure animation children comedy fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),comedy romance
2,1,6,4.0,964982224,Heat (1995),action crime thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),mystery thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",crime mystery thriller
...,...,...,...,...,...,...
100831,610,166534,4.0,1493848402,Split (2017),drama horror thriller
100832,610,168248,5.0,1493850091,John Wick: Chapter Two (2017),action crime thriller
100833,610,168250,5.0,1494273047,Get Out (2017),horror
100834,610,168252,5.0,1493846352,Logan (2017),action sci_fi


menghapus data duplikasi yang memiliki movieId yang sama. karena 1 film bisa muncul berkali-kali(karena user berbeda dalam ratings), sedangkan kita hanya ingin satu data unik untuk per film

In [15]:
movie_content_df = merged_df[['movieId', 'title', 'genres']].drop_duplicates('movieId')
movie_content_df.reset_index(drop=True, inplace=True)
movie_content_df.head()

,movieId,title,genres
0,1,Toy Story (1995),adventure animation children comedy fantasy
1,3,Grumpier Old Men (1995),comedy romance
2,6,Heat (1995),action crime thriller
3,47,Seven (a.k.a. Se7en) (1995),mystery thriller
4,50,"Usual Suspects, The (1995)",crime mystery thriller


In [16]:
#Mengkonversi data series 'movieID' menjadi dalam bentuk list
movie_id = movie_content_df['movieId'].tolist()

#mengkonversi data series 'title' menjadi dalam bentuk list
movie_title = movie_content_df['title'].tolist()

#Mengkonversi data series 'genres' menjadi dalam bentk list
movie_genres = movie_content_df['genres'].tolist()

print(len(movie_id))
print(len(movie_title))
print(len(movie_genres))

9724
9724
9724


setelah itu buat dictionary untuk menentukan pasangan key-value pada data movie_id, movie_title, dan movie_genres yang telah kita siapkan sebelumnya.

In [17]:
movie_new = pd.DataFrame ({
    'id' : movie_id,
    'movie_title' : movie_title,
    'genres': movie_genres
})

movie_new

,id,movie_title,genres
0,1,Toy Story (1995),adventure animation children comedy fantasy
1,3,Grumpier Old Men (1995),comedy romance
2,6,Heat (1995),action crime thriller
3,47,Seven (a.k.a. Se7en) (1995),mystery thriller
4,50,"Usual Suspects, The (1995)",crime mystery thriller
...,...,...,...
9719,160341,Bloodmoon (1997),action thriller
9720,160527,Sympathy for the Underdog (1971),action crime drama
9721,160836,Hazard (2005),action drama thriller
9722,163937,Blair Witch (2016),horror thriller


In [18]:
print(sorted(movie_new['genres'].unique()))

['(no genres listed)', 'action', 'action adventure', 'action adventure animation', 'action adventure animation children', 'action adventure animation children comedy', 'action adventure animation children comedy fantasy', 'action adventure animation children comedy imax', 'action adventure animation children comedy romance', 'action adventure animation children comedy sci_fi', 'action adventure animation children comedy sci_fi imax', 'action adventure animation children comedy western', 'action adventure animation children fantasy', 'action adventure animation children fantasy sci_fi', 'action adventure animation children sci_fi', 'action adventure animation comedy', 'action adventure animation comedy crime mystery', 'action adventure animation comedy fantasy mystery sci_fi', 'action adventure animation comedy fantasy sci_fi', 'action adventure animation comedy sci_fi', 'action adventure animation comedy thriller', 'action adventure animation crime fantasy', 'action adventure animation

# **Model Development dengan Content Based Filtering**


In [19]:
df = movie_new
df.sample(5)

,id,movie_title,genres
7096,4942,"Angry Red Planet, The (1959)",sci_fi
6402,3833,"Brain That Wouldn't Die, The (1962)",horror sci_fi
5455,162968,Kizumonogatari Part 1: Tekketsu (2016),action animation mystery
5158,26078,Advise and Consent (1962),drama
2022,139642,Southpaw (2015),action drama


# **TF-IDF Vectorizer**

In [20]:
print(sorted(df['genres'].unique()))

['(no genres listed)', 'action', 'action adventure', 'action adventure animation', 'action adventure animation children', 'action adventure animation children comedy', 'action adventure animation children comedy fantasy', 'action adventure animation children comedy imax', 'action adventure animation children comedy romance', 'action adventure animation children comedy sci_fi', 'action adventure animation children comedy sci_fi imax', 'action adventure animation children comedy western', 'action adventure animation children fantasy', 'action adventure animation children fantasy sci_fi', 'action adventure animation children sci_fi', 'action adventure animation comedy', 'action adventure animation comedy crime mystery', 'action adventure animation comedy fantasy mystery sci_fi', 'action adventure animation comedy fantasy sci_fi', 'action adventure animation comedy sci_fi', 'action adventure animation comedy thriller', 'action adventure animation crime fantasy', 'action adventure animation

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Inisialisasi TfidfVectorizer
tf = TfidfVectorizer()

# Melakukan perhitungan idf pada data genres
tf.fit(df['genres'])

# Mapping array dari fitur index integer ke fitur nama
tf.get_feature_names_out()


array(['action', 'adventure', 'animation', 'children', 'comedy', 'crime',
       'documentary', 'drama', 'fantasy', 'film_noir', 'genres', 'horror',
       'imax', 'listed', 'musical', 'mystery', 'no', 'romance', 'sci_fi',
       'thriller', 'war', 'western'], dtype=object)

selanjutnya kita akan melakukan fit dan transformasi kedalam bentuk matriks

In [22]:
#Melakukan fit lalu ditransformasikan ke bentuk matrix
tfidf_matrix = tf.fit_transform(df['genres'])

#melihat ukuran matrix tfidf
tfidf_matrix.shape

(9724, 22)

Untuk menghasilkan vektor tf-idf dalam bentuk matriks, kita menggunakan fungsi todense()

In [23]:
# Mengubah vektor tf-idf dalam bentuk matriks dengan fungsi todense()
tfidf_matrix.todense()

matrix([[0.        , 0.41681721, 0.51634045, ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.54896727, 0.        , 0.        , ..., 0.54222422, 0.        ,
         0.        ],
        ...,
        [0.64123095, 0.        , 0.        , ..., 0.63335461, 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.62477687, 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]])

kemudian mengubah matriks Tf_IDF menjadi sebuah DataFrame yang lebih mudah dibaca

In [24]:
pd.DataFrame(
    tfidf_matrix.todense(),
    columns=tf.get_feature_names_out(),
    index=df.movie_title
).sample(21, axis=1).sample(10, axis=0)

,film_noir,comedy,imax,documentary,adventure,western,musical,horror,war,romance,...,crime,children,animation,thriller,genres,fantasy,mystery,action,sci_fi,no
movie_title,,,,,,,,,,,,,,,,,,,,,
China Blue (2005),0.0,0.000000,0.0,1.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0
"Baby-Sitters Club, The (1995)",0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,1.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0
Captain America (1990),0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.569680,0.000000,...,0.0,0.0,0.000000,0.354702,0.0,0.473871,0.0,0.359114,0.442871,0.0
Intervista (1987),0.0,0.734280,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0
"Company, The (2003)",0.0,0.000000,0.0,0.0,0.000000,0.0,0.924337,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0
Fat Man and Little Boy (1989),0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0
Everest (2015),0.0,0.000000,0.0,0.0,0.689318,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.597955,0.0,0.000000,0.0,0.000000,0.000000,0.0
The Madagascar Penguins in a Christmas Caper (2005),0.0,0.460026,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.887905,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0
Rambo (Rambo 4) (2008),0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.713116,0.000000,...,0.0,0.0,0.000000,0.444011,0.0,0.000000,0.0,0.449532,0.000000,0.0


Pada tahap sebelumnya, kita telah berhasil mengidentifikasi korelasi antara movie dengan genre. Sekarang, kita akan menghitung derajat kesamaan (similarity degree) dengan teknik cosine similarity.

In [25]:
from sklearn.metrics.pairwise import cosine_similarity

#Menghitung cosine similarity pada matrix tf-idf
cosine_sim = cosine_similarity(tfidf_matrix)
cosine_sim

array([[1.        , 0.15262722, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.15262722, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.69543501, 0.33876915,
        0.        ],
       ...,
       [0.        , 0.        , 0.69543501, ..., 1.        , 0.39570531,
        0.        ],
       [0.        , 0.        , 0.33876915, ..., 0.39570531, 1.        ,
        0.78080334],
       [0.        , 0.        , 0.        , ..., 0.        , 0.78080334,
        1.        ]])

Pada tahapan ini, kita menghitung cosine similarity dataframe tfidf_matrix yang kita peroleh pada tahapan sebelumnya. Dengan satu baris kode untuk memanggil fungsi cosine similarity dari library sklearn, kita telah berhasil menghitung kesamaan (similarity) antar movie.


Selanjutnya, mari kita lihat matriks kesamaan setiap movie dengan menampilkan nama film dalam 5 sampel kolom (axis = 1) dan 10 sampel baris (axis=0).

In [26]:
#Membuat dataframe dari variabel cosine_sim dengan baris dan kolom berupa nama resto

cosine_sim_df = pd.DataFrame(cosine_sim, index=df['movie_title'], columns=df['movie_title'] )

#Melihat similarity matrix pada setiap movie
cosine_sim_df.sample(5, axis=1). sample(10, axis=0)

movie_title,Ashes of Time (Dung che sai duk) (1994),Puss in Boots: The Three Diablos (2012),Contagion (2011),Shot Caller (2017),"Burrowers, The (2008)"
movie_title,,,,,
Excess Baggage (1997),0.000000,0.000000,0.0,0.000000,0.0
Cheech and Chong's Up in Smoke (1978),0.000000,0.460026,0.0,0.000000,0.0
"Rugrats Movie, The (1998)",0.000000,0.755211,0.0,0.000000,0.0
"Young and the Damned, The (Olvidados, Los) (1950)",0.503700,0.000000,0.0,0.690379,0.0
Undercover Brother (2002),0.000000,0.460026,0.0,0.000000,0.0
"Men, Women & Children (2014)",0.678847,0.337788,0.0,0.236065,0.0
A Dog's Purpose (2017),0.678847,0.337788,0.0,0.236065,0.0
Losing Isaiah (1995),1.000000,0.000000,0.0,0.347744,0.0
Private School (1983),0.000000,0.460026,0.0,0.000000,0.0


#### **Rekomendasi**

In [27]:
def movie_recommendations(movie_title, similarity_data=cosine_sim_df, items=df[['movie_title', 'genres']], k=5):
    index = similarity_data.loc[:,movie_title].to_numpy().argpartition(
        range(-1, -k, -1))
    closest = similarity_data.columns[index[-1:-(k+2):-1]]
    closest = closest.drop(movie_title, errors='ignore')
    return pd.DataFrame(closest).merge(items).head(k)

In [28]:
df[df.movie_title.eq('Waterboy, The (1998)')]

,id,movie_title,genres
1598,2335,"Waterboy, The (1998)",comedy


In [29]:
#Mendapatkan rekomendasi movie yang mirip dengan 'Waterboy, The (1998)'
movie_recommendations('Waterboy, The (1998)')

,movie_title,genres
0,Private Benjamin (1980),comedy
1,Judd Apatow: The Return (2017),comedy
2,Dad's Army (2016),comedy
3,Craig Ferguson: Tickle Fight (2017),comedy
4,Dad's Army (1971),comedy


Berdasarkan hasil rekomendasi film yang memiliki genre yang sama dengan 'Waterboy, The (1998)' adalah :
1. Private Benjamin (1980)	comedy
2. Judd Apatow: The Return (2017)	comedy
3. Dad's Army (2016)	comedy
4. Craig Ferguson: Tickle Fight (2017)	comedy
5. Dad's Army (1971)	comedy

# Model Collaborative Filtering

model rekomendasi berbasis Collaborative Filtering (CF), khususnya item-based collaborative filtering dengan pendekatan Pearson Correlation sebagai ukuran kemiripan antar item (film).

In [30]:
merged_df

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),adventure animation children comedy fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),comedy romance
2,1,6,4.0,964982224,Heat (1995),action crime thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),mystery thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",crime mystery thriller
...,...,...,...,...,...,...
100831,610,166534,4.0,1493848402,Split (2017),drama horror thriller
100832,610,168248,5.0,1493850091,John Wick: Chapter Two (2017),action crime thriller
100833,610,168250,5.0,1494273047,Get Out (2017),horror
100834,610,168252,5.0,1493846352,Logan (2017),action sci_fi


menghitung jumlah review atau rating dari masing-masing judul film yang diberikan oleh user

In [31]:
comment_counts = merged_df["title"].value_counts().reset_index()
comment_counts.columns = ['title', 'count_user']
comment_counts.head(5)

,title,count_user
0,Forrest Gump (1994),329
1,"Shawshank Redemption, The (1994)",317
2,Pulp Fiction (1994),307
3,"Silence of the Lambs, The (1991)",279
4,"Matrix, The (1999)",278


filter film berdasarkan popularitas atau jumlah user rating

In [32]:
popular_movies = comment_counts[comment_counts["count_user"] >= 50]['title'].tolist()
popular_movies

['Forrest Gump (1994)',
 'Shawshank Redemption, The (1994)',
 'Pulp Fiction (1994)',
 'Silence of the Lambs, The (1991)',
 'Matrix, The (1999)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Jurassic Park (1993)',
 'Braveheart (1995)',
 'Terminator 2: Judgment Day (1991)',
 "Schindler's List (1993)",
 'Fight Club (1999)',
 'Toy Story (1995)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'American Beauty (1999)',
 'Usual Suspects, The (1995)',
 'Seven (a.k.a. Se7en) (1995)',
 'Independence Day (a.k.a. ID4) (1996)',
 'Apollo 13 (1995)',
 'Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)',
 'Lord of the Rings: The Fellowship of the Ring, The (2001)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Godfather, The (1972)',
 'Fugitive, The (1993)',
 'Batman (1989)',
 'Saving Private Ryan (1998)',
 'Lord of the Rings: The Two Towers, The (2002)',
 'Lord of the Rings: The Return of the King, The (2003)',
 'Aladdin (1992)',
 'Fargo (1996)',


sekarang kita akan membuat matriks pengguna vs film. Dimana :
- baris = userId
- kolom = title (judul film)
- Nilai = rating yang diberikan user ke film tersebut

In [33]:
# Filter data hanya untuk film-film dalam cut_movies
filtered_df = merged_df[merged_df['title'].isin(popular_movies)]

# Buat matriks user vs film, isi nilainya adalah rating
user_movie_matrix = filtered_df.pivot_table(index='userId', columns='title', values='rating')

# Lihat hasil awal
user_movie_matrix.head()


title,10 Things I Hate About You (1999),12 Angry Men (1957),2001: A Space Odyssey (1968),28 Days Later (2002),300 (2007),"40-Year-Old Virgin, The (2005)",A.I. Artificial Intelligence (2001),"Abyss, The (1989)",Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),...,Willy Wonka & the Chocolate Factory (1971),"Wizard of Oz, The (1939)","Wolf of Wall Street, The (2013)",X-Men (2000),X-Men: The Last Stand (2006),X2: X-Men United (2003),You've Got Mail (1998),Young Frankenstein (1974),Zombieland (2009),Zoolander (2001)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,5.0,5.0,NaN,5.0,NaN,NaN,NaN,5.0,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN
4,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
user_movie_matrix.shape

(606, 450)

In [35]:
user_movie_matrix.columns

Index(['10 Things I Hate About You (1999)', '12 Angry Men (1957)',
       '2001: A Space Odyssey (1968)', '28 Days Later (2002)', '300 (2007)',
       '40-Year-Old Virgin, The (2005)', 'A.I. Artificial Intelligence (2001)',
       'Abyss, The (1989)', 'Ace Ventura: Pet Detective (1994)',
       'Ace Ventura: When Nature Calls (1995)',
       ...
       'Willy Wonka & the Chocolate Factory (1971)',
       'Wizard of Oz, The (1939)', 'Wolf of Wall Street, The (2013)',
       'X-Men (2000)', 'X-Men: The Last Stand (2006)',
       'X2: X-Men United (2003)', 'You've Got Mail (1998)',
       'Young Frankenstein (1974)', 'Zombieland (2009)', 'Zoolander (2001)'],
      dtype='object', name='title', length=450)

- membuat similarity matrix menggunakan Pearson Correlation:
Punya matriks user-film
Baris = userId
Kolom = judul film
Nilai = rating user ke film tersebut

- Hitung korelasi antar film
Kita hitung Pearson correlation antar kolom (film).
Dengan ini, kita tahu seberapa mirip pola rating antar dua film.

- Gunakan matriks korelasi ini untuk rekomendasi
Kalau user suka film A, rekomendasikan film-film yang punya korelasi tinggi dengan film A.

menghitung matrix person correlation antar film. korelaasi dihitung hanya untuk film yang memiliki minimal nilai 5, untuk user yang sama-sama memberikan rating

In [36]:
movie_similarity = user_movie_matrix.corr(method='pearson', min_periods=5)

Nilai korelasi:

- (1) artinya pola rating kedua film sangat mirip.

- (0) artinya tidak ada korelasi.

- (-1) artinya pola rating berlawanan (jika suka satu, biasanya tidak suka yang lain).



In [37]:
print(movie_similarity.head())

title                              10 Things I Hate About You (1999)  \
title                                                                  
10 Things I Hate About You (1999)                       1.000000e+00   
12 Angry Men (1957)                                    -4.382839e-17   
2001: A Space Odyssey (1968)                           -2.535161e-01   
28 Days Later (2002)                                    1.079482e-01   
300 (2007)                                              3.394157e-01   

title                              12 Angry Men (1957)  \
title                                                    
10 Things I Hate About You (1999)        -4.382839e-17   
12 Angry Men (1957)                       1.000000e+00   
2001: A Space Odyssey (1968)              4.022474e-02   
28 Days Later (2002)                     -9.787004e-03   
300 (2007)                               -2.860686e-01   

title                              2001: A Space Odyssey (1968)  \
title                

 mencari film yang paling mirip dengan film yang dipilih (movie_title). Pertama, fungsi mengambil skor korelasi antara film lain dengan film tersebut. Selanjutnya, film yang sama dengan movie_title dihapus agar tidak muncul sebagai rekomendasi. Setelah itu, skor korelasi diurutkan dari yang tertinggi ke terendah, dan akhirnya fungsi mengembalikan 5 film teratas sebagai rekomendasi

In [38]:
def recommend_movies(movie_title, movie_similarity, top_n=5):
    if movie_title not in movie_similarity.columns:
        print("Film tidak ditemukan di data.")
        return

    sim_scores = movie_similarity[movie_title].dropna()
    sim_scores = sim_scores.drop(labels=[movie_title])
    top_movies = sim_scores.sort_values(ascending=False).head(top_n)

    print(f"Rekomendasi film mirip dengan '{movie_title}':\n")
    for i, (title, score) in enumerate(top_movies.items(), start=1):
        print(f"{i}. {title} (Skor kemiripan: {score:.3f})")


In [39]:
# Mencari rekomendasi untuk film "Forrest Gump (1994)"
recommendations = recommend_movies('Forrest Gump (1994)', movie_similarity)
print(recommendations)

Rekomendasi film mirip dengan 'Forrest Gump (1994)':

1. Mr. Holland's Opus (1995) (Skor kemiripan: 0.652)
2. Pocahontas (1995) (Skor kemiripan: 0.550)
3. Grumpier Old Men (1995) (Skor kemiripan: 0.535)
4. Caddyshack (1980) (Skor kemiripan: 0.520)
5. Few Good Men, A (1992) (Skor kemiripan: 0.517)
None


berdasarkan rekomendasi kelima film tersebut punya pola rating yang cukup mirip dengan "Forrest Gump (1994)".

Korelasi di atas 0.5 cukup tinggi untuk rekomendasi film.

Jadi, jika user suka "Forrest Gump (1994)", sistem menyarankan film-film tersebut.